In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LassoLars, LinearRegression, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

import wrangle
import explore

import warnings
warnings.filterwarnings("ignore")

# Project Objective

### The goal of this project was to predict property tax assessed values of Single Family Properties located in three counties in southern California that had a transaction in 2017. 